In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
from pyspark.streaming import StreamingContext
from pyspark.ml import PipelineModel

In [6]:
globals()['models_loaded'] = False
globals()['my_model'] = None

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()

     # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = PipelineModel.load('LRmodel') # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model (uncomment below):
    
    df_result = globals()['my_model'].transform(df)
    df_result.show()

In [7]:
ssc = StreamingContext(sc, 10)

In [8]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [9]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2023-05-28 17:25:30 =========
+------+-----+---------+--------------------+
|app_id|label|review_id|         review_text|
+------+-----+---------+--------------------+
|824600|    1|139152723|A very fun and fu...|
|824600|    1|139152686|Easily the best b...|
+------+-----+---------+--------------------+

+------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|app_id|label|review_id|         review_text|              tokens|     filtered_tokens|            features|       rawPrediction|         probability|prediction|
+------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|824600|    1|139152723|A very fun and fu...|[a, very, fun, an...|  [fun, funny, game]|(127099,[0,5,567]...|[-4.4841762102359...|[0.01116022481398...|       1.0|
|824600|    1|139152686|Easily the 

========= 2023-05-28 17:26:50 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1304930|    0|139152036|Dumb “game as a s...|
|1304930|    1|139151782|The Outlast Multi...|
|1304930|    1|139151444|"If You Ain't Gui...|
|1304930|    1|139151268|The game is not t...|
+-------+-----+---------+--------------------+

+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| app_id|label|review_id|         review_text|              tokens|     filtered_tokens|            features|       rawPrediction|         probability|prediction|
+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|1304930|    0|139152036|Dumb “game as a s...|[dumb, “game, as,...|[dumb, “game, ser...|(12709

In [10]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| app_id|label|review_id|         review_text|              tokens|     filtered_tokens|            features|       rawPrediction|         probability|prediction|
+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|2141910|    0|139153316|I hate this game ...|[i, hate, this, g...|[hate, game, keep...|(127099,[0,18,23,...|[-1.4309120001859...|[0.19295662381282...|       1.0|
|2141910|    0|139153213|It feels like Wiz...|[it, feels, like,...|[feels, like, wiz...|(127099,[0,1,2,3,...|[-29.219850156610...|[2.04164540189928...|       1.0|
|2141910|    1|139152451|I can't even smel...|[i, can't, even, ...|[even, smell, opp...|(127099,[12,9691]...|[-5.1220625412634...|[0.